In [1]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import time
import models
import gdown

from scipy.stats import nbinom,norm,truncnorm
from scipy.integrate import trapezoid as trapz

In [2]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=0&single=true&output=csv"
df = pd.read_csv(url)

In [3]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner'])
losers=np.array(df['Loser'])
loser_scores=np.array(df['Points'])
names = pd.concat([df['Winner'], df['Loser']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = np.array([dict_to_id[i] for i in winners])
loser_ids = np.array([dict_to_id[i] for i in losers])

   Winner   Loser  Points
0    Fedo   Mauro       1
1    Fedo   Idris       3
2   Mauro   Idris       6
3    Fedo   Idris       5
4   Mauro   Idris       8
5    Fedo   Mauro       5
6   Mauro   Frago       7
7   Mauro   Frago       7
8   Frago   Mauro       6
9   Frago   Idris       9
10  Frago   Idris       9
11  Frago   Idris      10
12  Frago   Idris       8
13  Mauro  Satvik       7
14   Fedo   Idris       4
15   Fedo   Idris       9
16   Fedo  Satvik      10
17   Fedo  Satvik       2
18   Fedo  Satvik       1
19   Fedo  Satvik       1


In [12]:
winner_ids=torch.Tensor(winner_ids).int()
loser_ids=torch.Tensor(loser_ids).int()
loser_scores=torch.Tensor(loser_scores).int()

In [13]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(models.pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=100, warmup_steps=10)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Sample: 100%|██| 110/110 [00:01, 67.73it/s, step size=2.40e-01, acc. prob=0.319]


In [14]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(ids)):
    player = f'Player {i+1}: {dict_to_name[i]}'
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

Player 1: Fedo
Posterior mean of strength: 2.2895824909210205
Posterior std dev of strength: 0.4013243615627289

Player 2: Mauro
Posterior mean of strength: 0.9741412401199341
Posterior std dev of strength: 0.7356550097465515

Player 3: Frago
Posterior mean of strength: 1.2298502922058105
Posterior std dev of strength: 0.2668052911758423

Player 4: Idris
Posterior mean of strength: 1.0444647073745728
Posterior std dev of strength: 0.2371239811182022

Player 5: Satvik
Posterior mean of strength: 0.8340847492218018
Posterior std dev of strength: 0.3003476560115814



## Adding Gibbs sampling

In [15]:
#just in case you run this model after the previous one
if isinstance(winner_ids, torch.Tensor):
    winner_ids=winner_ids.numpy()
    loser_ids=loser_ids.numpy()
    loser_scores=loser_scores.numpy()

In [4]:
g=models.gibbs_model(winner_ids, loser_ids, loser_scores,names=names)

In [5]:
g.posterior_sampling(n_iterations=50, warmup=10)

Progress: [################################################# ] 98%

In [6]:
g.print_table()

Player 1
Posterior mean of strength: 0.886335464638741
Posterior std dev of strength: 0.08799094281178609

Player 2
Posterior mean of strength: 0.7075505877969136
Posterior std dev of strength: 0.10585778442282497

Player 3
Posterior mean of strength: 0.6427014682357448
Posterior std dev of strength: 0.12652024346292817

Player 4
Posterior mean of strength: 0.49967556170490257
Posterior std dev of strength: 0.07929602751911267

Player 5
Posterior mean of strength: 0.350905408395462
Posterior std dev of strength: 0.09104930285161758

Player 1 has 70.6% of probability of winning against Player 2
Player 1 has 76.1% of probability of winning against Player 3
Player 1 has 89.5% of probability of winning against Player 4
Player 1 has 97.6% of probability of winning against Player 5
Player 2 has 60.3% of probability of winning against Player 3
Player 2 has 77.4% of probability of winning against Player 4
Player 2 has 92.60000000000001% of probability of winning against Player 5
Player 3 has 6